# Olist : Construire un modèle de segmentation client

## Contexte

Olist est une place de marché en ligne qui permet à des e-marchands de proposer leurs produits aux internautes Brésiliens.
Afin d'optimiser les campagnes de communication, il est nécessaire de bien adapter le discours à chaque catégorie de clients, et donc de bien comprendre chaque typologie de clients. Pour répondre à cette problématique, il faut définir une stratégie de segmentation des clients efficace. 

L'équipe marketing utilisera le modèle de segmentation. Ils auront besoin d'un outil fiable, rapide et facile à utiliser.


## Chargement des modules du projet

Afin de simplifier le Notebook, les fonctions utiles sont placées dans le dossier [src/](../src/).

Nous allons utiliser le langage [Python](https://www.python.org/about/gettingstarted/), et présenter ici le code, les résultats et l'analyse sous forme de [Notebook JupyterLab](https://jupyterlab.readthedocs.io/en/stable/getting_started/overview.html).

Nous allons aussi utiliser les bibliothèques usuelles d'exploration et analyse de données, afin d'améliorer la simplicité et la performance de notre code :

-   [NumPy](https://numpy.org/doc/stable/user/quickstart.html) et [Pandas](https://pandas.pydata.org/docs/user_guide/index.html) : effectuer des calculs scientifiques (statistiques, algèbre, ...) et manipuler des séries et tableaux de données volumineuses et complexes
-   [scikit-learn](https://scikit-learn.org/stable/getting_started.html) : pour modéliser les données 
-   [Plotly](https://plotly.com/python/getting-started/) : générer des graphiques lisibles, interactifs et pertinents


In [ ]:
# System modules
import os
import sys
from time import time

# Append source directory to system path
src_path = os.path.abspath(os.path.join("../src"))
if src_path not in sys.path:
    sys.path.append(src_path)

# Helper functions
import data.helpers as data_helpers
import features.helpers as feat_helpers
import visualization.helpers as vis_helpers
import models.helpers as models_helpers

# numpy and pandas for data manipulation
import numpy as np
import pandas as pd

# matplotlib and seaborn for plotting
import plotly.express as px
import plotly.graph_objects as go


# Accelerate the development cycle
SAMPLE_FRAC: float = 0.5

# Prevent excessive memory usage used by plotly
DRAW_PLOTS: bool = True


## Chargement des données

Nous allons télécharger et dé-zipper les fichiers CSV, puis les charger en mémoire, en prenant soin d'utiliser le bon type pour chaque variable.

Le schema de données nous est donné : 

![](https://i.imgur.com/HRhd2Y0.png)


In [ ]:
# Download and unzip CSV files
!cd .. && make dataset && cd notebooks


In [ ]:
# Load customers data
customers_df = pd.read_csv(
    "../data/raw/olist_customers_dataset.csv",
    dtype={
        # Nominal qualitative data
        "customer_id": "category",
        "customer_unique_id": "category",
        "customer_city": "category",
        "customer_state": "category",
        "customer_zip_code_prefix": "category",
    },
)

# Load geolocation data
geolocation_df = pd.read_csv(
    "../data/raw/olist_geolocation_dataset.csv",
    dtype={
        # Nominal qualitative data
        "geolocation_zip_code_prefix": "category",
        "geolocation_city": "category",
        "geolocation_state": "category",
        # Continuous quantitative data
        "geolocation_lat": float,
        "geolocation_lng": float,
    },
)

# Load order items data
order_items_df = pd.read_csv(
    "../data/raw/olist_order_items_dataset.csv",
    dtype={
        # Nominal qualitative data
        "order_id": "category",
        "order_item_id": "category",
        "product_id": "category",
        "seller_id": "category",
        # Date data
        "shipping_limit_date": str,
        # Continuous quantitative data
        "price": float,
        "freight_value": float,
    },
    parse_dates=["shipping_limit_date"],
)

# Load order payments
order_payments_df = pd.read_csv(
    "../data/raw/olist_order_payments_dataset.csv",
    dtype={
        # Nominal qualitative data
        "order_id": "category",
        "payment_type": "category",
        # Discrete quantitative data
        "payment_sequential": int,
        "payment_installments": int,
        # Continuous quantitative data
        "payment_value": float,
    },
)

# Load order reviews
order_reviews_df = pd.read_csv(
    "../data/raw/olist_order_reviews_dataset.csv",
    dtype={
        # Nominal qualitative data
        "review_id": "category",
        "order_id": "category",
        # Discrete quantitative data
        "review_score": int,
        # Text data
        "review_comment_title": str,
        "review_comment_message": str,
        # Date data
        "review_creation_date": str,
        "review_answer_timestamp": str,
    },
    parse_dates=["review_creation_date", "review_answer_timestamp"],
)

# Load orders data
orders_df = pd.read_csv(
    "../data/raw/olist_orders_dataset.csv",
    dtype={
        # Nominal qualitative data
        "order_id": "category",
        "customer_id": "category",
        "order_status": "category",
        # Date data
        "order_purchase_timestamp": str,
        "order_approved_at": str,
        "order_delivered_carrier_date": str,
        "order_delivered_customer_date": str,
        "order_estimated_delivery_date": str,
    },
    parse_dates=[
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date",
    ],
)

# Load products data
products_df = pd.read_csv(
    "../data/raw/olist_products_dataset.csv",
    dtype={
        # Nominal qualitative data
        "product_id": "category",
        "product_category_name": "category",
        # Discrete quantitative data
        # Nullable : https://pandas.pydata.org/pandas-docs/stable/user_guide/gotchas.html#support-for-integer-na
        "product_name_lenght": pd.Int64Dtype(),
        "product_description_lenght": pd.Int64Dtype(),
        "product_photos_qty": pd.Int64Dtype(),
        # Continuous quantitative data
        "product_weight_g": float,
        "product_length_cm": float,
        "product_height_cm": float,
        "product_width_cm": float,
    },
)

# Load sellers data
sellers_df = pd.read_csv(
    "../data/raw/olist_sellers_dataset.csv",
    dtype={
        # Nominal qualitative data
        "seller_id": "category",
        "seller_city": "category",
        "seller_state": "category",
        "seller_zip_code_prefix": "category",
    },
)

# Load category name translation data
category_translation_df = pd.read_csv(
    "../data/raw/product_category_name_translation.csv"
)


### Construction des variables RFM (Recency, Frequency, Monetary)

La segmentation des clients selon les variables RFM (Recency, Frequency, Monetary) est une méthode classique en marketing.

Pour un client donné, nous définissons les variables comme :

-   Recency : nombre de jours depuis le dernier achat du client
-   Frequency : nombre total d'achats du client
-   Monetary : montant moyen des achats du client


In [ ]:
# Compute the sum of the payment values for each order
merged_orders_payments_df = (
    orders_df[["order_id", "customer_id", "order_purchase_timestamp"]]
    .merge(
        order_payments_df[["order_id", "payment_value"]],
        how="left",
        left_on="order_id",
        right_on="order_id",
        validate="1:m",
    )
    .groupby("order_id")
    .agg(
        customer_id=("customer_id", "first"),
        order_purchase_timestamp=("order_purchase_timestamp", "first"),
        payment_value=("payment_value", "sum"),  # Total order value
    )
)

# Compute the RFM variables
rfm_df = (
    customers_df[["customer_id", "customer_unique_id"]]
    .merge(
        merged_orders_payments_df,
        how="left",
        left_on="customer_id",
        right_on="customer_id",
        validate="1:1",
    )
    .groupby("customer_unique_id")
    .agg(
        recency=(  # Recency : last purchase date
            "order_purchase_timestamp",
            "max",
        ),
        frequency=(  # Frequency : total number of purchases
            "customer_id",
            "count",
        ),
        monetary=("payment_value", "mean"),  # Monetary : average purchase value
    )
)

# Recency : Transform last purchase date to days since last purchase (from the very last order recorded)
rfm_df["recency"] = (
    (rfm_df["recency"] - rfm_df["recency"].max()) / np.timedelta64(1, "D")
).values

# Reduce memory usage
rfm_df = data_helpers.reduce_dataframe_memory_usage(rfm_df)

rfm_df.describe(include="all", datetime_is_numeric=True)


Il n'y a pas de valeurs vides et toutes les valeurs semblent "possibles" (pas de valeur impossible : une date d'achat dans le futur, ou un montant négatif...).

Observons la distribution de nos variables.


#### Distributions des variables

Observons les points de données dans l'espace RFM.


In [ ]:
# Plot the RFM variables scatter matrix to see their correlation
if DRAW_PLOTS:
    fig = px.scatter_3d(
        rfm_df.sample(  # sample of the data for performances reasons
            n=1000,
            random_state=42,
        ),
        x="recency",
        y="frequency",
        z="monetary",
        title="Clients in RFM space",
        opacity=0.5,
        width=1200,
        height=800,
    )
    fig.show()


In [ ]:
# Plot the RFM variables histograms and boxes with Plotly
if DRAW_PLOTS:
    for col in rfm_df.columns:
        fig = px.histogram(
            rfm_df[col],
            marginal="box",
            width=800,
        )
        fig.show()


Nous voyons que le nombre d'achats a tendance à augmenter dans le temps. La plupart (93%) des clients achètent n'ont effectué qu'un seul achat. Le panier moyen est de 161 Reals.

In [ ]:
# Plot the RFM variables scatter matrix to see their correlation
if DRAW_PLOTS:
    fig = px.scatter_matrix(
        rfm_df.sample(  # sample of the data for performances reasons
            n=1000,
            random_state=42,
        ),
        width=1200,
        height=800,
    )
    fig.update_traces(
        diagonal_visible=False,
        showupperhalf=False,
    )
    fig.show()


Nous voyons que les clients "fréquents" ont tendance à être plus "récents", ce qui est plutôt logique (le second achat est forcément postérieur au premier...).


In [ ]:
# Plot the Recency x Monetary trendline to see if there is a correlation
if DRAW_PLOTS:
    fig = px.scatter(
        rfm_df.sample(  # sample of the data for performances reasons
            n=1000,
            random_state=42,
        ),
        x="recency",
        y="monetary",
        labels={
            "recency": "Recency : days since last purchase",
            "frequency": "Frequency : total number of purchases",
            "monetary": "Monetary : average purchase value",
        },
        # color="frequency",
        # size="frequency",
        trendline="ols",
        trendline_color_override="red",
        marginal_x="histogram",
        marginal_y="histogram",
        width=1200,
        height=800,
    )
    fig.show()


### Scaling, Cleaning & Sampling

Nous voyons que nos variables ont des ordres de grandeur très différents et qu'il y a quelques valeurs "extrêmes" qui pourront perturber l'entraînement de nos modèles. Nous allons donc transformer nos données pour les rendre plus facilement exploitables.


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler



# # Scale the RFM variables
# scaler = StandardScaler()
# scaled_rfm_df = pd.DataFrame(
#     scaler.fit_transform(rfm_df), columns=rfm_df.columns
# )

# # Remove outliers and duplicates
# scaled_rfm_df = (
#     scaled_rfm_df[scaled_rfm_df < 20]  # Remove outliers
#     .dropna()  # Remove rows with NaN values
#     .drop_duplicates()  # Remove duplicates
#     .sample(frac=SAMPLE_FRAC, random_state=42)  # Sample the data
# )

# # Inverse-transform the data to get the same sample of original values
# rfm_df = pd.DataFrame(
#     scaler.inverse_transform(scaled_rfm_df), columns=scaled_rfm_df.columns
# )

# rfm_df.describe(include="all", datetime_is_numeric=True)


In [ ]:
def rfm_fit(
    df: pd.DataFrame,
) -> tuple[StandardScaler, MinMaxScaler]:
    df = df.copy()
    df["recency"] = -df["recency"]  # Invert the recency to have positive values
    log_df = np.log1p(df.astype(float))
    standard_scaler = StandardScaler().fit(log_df)
    minmax_scaler = MinMaxScaler().fit(standard_scaler.transform(log_df))
    return standard_scaler, minmax_scaler


def rfm_transform(
    df: pd.DataFrame,
    standard_scaler: StandardScaler,
    minmax_scaler: MinMaxScaler,
) -> pd.DataFrame:
    df = df.copy()
    df["recency"] = -df["recency"]  # Invert the recency to have positive values
    return pd.DataFrame(
        minmax_scaler.transform(
            standard_scaler.transform(np.log1p(df.astype(float)))
        ),
        columns=df.columns,
    )


def rfm_inverse_transform(
    df: pd.DataFrame,
    minmax_scaler: MinMaxScaler,
    standard_scaler: StandardScaler,
) -> pd.DataFrame:
    df = df.copy()
    df = np.expm1(
        pd.DataFrame(
            standard_scaler.inverse_transform(
                minmax_scaler.inverse_transform(df)
            ),
            columns=df.columns,
        )
    )
    df["recency"] = -df["recency"]  # Invert the recency to have negative values
    return df


standard_scaler, minmax_scaler = rfm_fit(rfm_df)


In [ ]:
sampled_rfm_df = rfm_df.sample(frac=0.1, random_state=42)
sampled_rfm_df.describe(include="all", datetime_is_numeric=True)


In [ ]:
fig = px.scatter_3d(
    sampled_rfm_df,
    x="recency",
    y="frequency",
    z="monetary",
    title="Clients in RFM space",
    opacity=0.5,
    width=1200,
    height=800,
)
fig.show()


In [ ]:
scaled_rfm_df = rfm_transform(
    sampled_rfm_df,
    standard_scaler=standard_scaler,
    minmax_scaler=minmax_scaler,
)  # Scale the RFM variables
scaled_rfm_df.describe(include="all", datetime_is_numeric=True)


In [ ]:
fig = px.scatter_3d(
    scaled_rfm_df,
    x="recency",
    y="frequency",
    z="monetary",
    title="Clients in RFM space",
    opacity=0.5,
    width=1200,
    height=800,
)
fig.show()


In [ ]:
unscaled_rfm_df = rfm_inverse_transform(
    scaled_rfm_df,
    minmax_scaler=minmax_scaler,
    standard_scaler=standard_scaler,
)  # UnScale the RFM variables
unscaled_rfm_df.describe(include="all", datetime_is_numeric=True)


In [ ]:
fig = px.scatter_3d(
    unscaled_rfm_df,
    x="recency",
    y="frequency",
    z="monetary",
    title="Clients in RFM space",
    opacity=0.5,
    width=1200,
    height=800,
)
fig.show()


Nos données sont maintenant prêtes à être exploitées.


### EDA & Visualization


#### Distances

Nous allons chercher à observer quelle est la répartition de la distance euclidienne entre nos clients dans l'espace RFM transformé. Cette information nous donnera une indication utile pour les modèles de segmentation utilisant le voisinage des points (comme DBSCAN).


In [ ]:
from sklearn.neighbors import NearestNeighbors


# Compute the nearest neighbors of each customer and their distance
if DRAW_PLOTS:
    knn = NearestNeighbors(n_neighbors=2).fit(scaled_rfm_df)
    distances, indices = knn.kneighbors(scaled_rfm_df)
    distances = np.sort(distances[:1000], axis=0)
    distances = distances[:, 1]

    # Plot the nearest neighbors distances
    fig = px.line(
        distances,
        labels={
            "index": "Couple of customers",
            "value": "Euclidian distance",
        },
        title="Customers distances in scaled RFM space",
        width=800,
    )
    fig.show()


En utilisant la méthode du coude, nous voyons que la distance euclidienne entre deux points est inférieure à 0.025 pour 95% des couples de points.


#### Dendrogram

Nous allons voir ici la hiérarchie des variables RFM. Ceci nous permettra d'anticiper un nombre "idéal" de clusters pour notre segmentation.


In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage


# Plot the hierarchical clustering of the scaled RFM variables
if DRAW_PLOTS:
    dendrogram(
        linkage(scaled_rfm_df.sample(frac=0.5, random_state=42), method="ward"),
        truncate_mode="level",
        p=2,
    )


Nous voyons qu'avec 4 clusters, nous sommes dans la zone la plus large entre deux nouveaux clusters. Une segmentation à 2, 5 ou 6 clusters seraient aussi possibles.


#### PCA (Principal Component Analysis)

Nous allons voir ici la projection des variables RFM sur les deux premières composantes principales. Ceci nous permettra de voir à quel point il sera difficile de segmenter nos clients.


In [ ]:
from sklearn.decomposition import PCA


if DRAW_PLOTS:
    pca = PCA(n_components=2, random_state=42)
    data_pca = pca.fit_transform(scaled_rfm_df)

    # Plot the data in the PCA space
    fig = px.scatter(
        x=data_pca[:1000, 0],
        y=data_pca[:1000, 1],
        trendline="ols",
        title="PCA 2D",
        opacity=0.5,
        width=1200,
        height=800,
        labels={"x": "PCA 1", "y": "PCA 2"},
    )

    # Plot the feature importances in the PCA space
    loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
    for i, feature in enumerate(rfm_df.columns):
        fig.add_shape(
            type="line",
            x0=0,
            y0=0,
            x1=loadings[i, 0],
            y1=loadings[i, 1],
            line=dict(color="red", width=3),
            name=feature,
        )
        fig.add_annotation(
            x=loadings[i, 0],
            y=loadings[i, 1],
            ax=0,
            ay=0,
            xanchor="center",
            yanchor="bottom",
            text=feature,
            name=feature,
        )

    fig.show()


Nous voyons que nos données ne semblent pas très facilement segmentables dans cet espace.


## Models evaluation


In [ ]:
from sklearn.base import ClusterMixin
from sklearn.metrics import (
    silhouette_score,  # higher is better : https://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient
    davies_bouldin_score,  # lower is better : https://scikit-learn.org/stable/modules/clustering.html#davies-bouldin-index
    calinski_harabasz_score,  # higher is better : https://scikit-learn.org/stable/modules/clustering.html#calinski-harabasz-index
)


models_results = pd.DataFrame(
    columns=[
        "model",
        "n_clusters",
        "labels",
        "cluster_centers",
        "inertia",
        "time",
        "silhouette_score",
        "davies_bouldin_score",
        "calinski_harabasz_score",
    ]
)


def process_model(
    model_class: ClusterMixin,
    model_args: dict,
    param_name: str,
    param_range: list,
    fit_df: pd.DataFrame,
    pred_df: pd.DataFrame,
    verbose: bool = False,
) -> dict:
    results = pd.DataFrame(
        columns=[
            "model",
            "n_clusters",
            "labels",
            "cluster_centers",
            "inertia",
            "time",
            "silhouette_score",
            "davies_bouldin_score",
            "calinski_harabasz_score",
        ]
    )

    for param_value in param_range:
        model_args[param_name] = param_value
        model = model_class(**model_args)

        if verbose:
            print(f">>> Model : { model }")

        if hasattr(model, "fit") and hasattr(model, "predict"):
            start_time = time()
            model.fit(fit_df)
            fit_time = (time() - start_time) / fit_df.shape[0]

            start_time = time()
            predicted_labels = model.predict(pred_df)
            pred_time = (time() - start_time) / pred_df.shape[0]

            fit_pred_time = fit_time + pred_time
        elif hasattr(model, "fit_predict"):
            start_time = time()
            predicted_labels = model.fit_predict(pred_df)
            fit_pred_time = time() - start_time
        else:
            raise ValueError(f"{model} is not a clustering model.")

        n_clusters = predicted_labels.max() + 1
        if verbose:
            print(f"Number of clusters : { n_clusters }")

        if not 1 < n_clusters < 11:
            continue

        result = {
            "model": str(model)
            .replace(", random_state=42", "")
            .replace(", n_jobs=-1", "")
            .replace("(random_state=42", "(")
            .replace("(n_jobs=-1", "("),
            "n_clusters": n_clusters,
            "labels": predicted_labels,
            "cluster_centers": model.cluster_centers_
            if hasattr(model, "cluster_centers_")
            else None,
            "inertia": model.inertia_ if hasattr(model, "inertia_") else None,
            "fit_pred_time": fit_pred_time,
            "silhouette_score": silhouette_score(
                pred_df, predicted_labels, random_state=42
            ),
            "davies_bouldin_score": davies_bouldin_score(
                pred_df, predicted_labels
            ),
            "calinski_harabasz_score": calinski_harabasz_score(
                pred_df, predicted_labels
            ),
        }

        if verbose:
            print(f"Score : { round(result['silhouette_score'], 3) }")

        results = results.append(result, ignore_index=True)

    return results


def plot_scores(results: pd.DataFrame) -> None:
    fig = px.line(
        results,
        x="model",
        y=[
            "silhouette_score",
            "davies_bouldin_score",
        ],
        title="Clustering models evaluation",
        markers=True,
        width=800,
    )
    fig.show()


def plot_clusters(model_name, pred_df, labels, cluster_centers=None) -> None:
    fig = px.scatter_3d(
        pred_df,
        x="recency",
        y="frequency",
        z="monetary",
        title=f"Clustering : { model_name }",
        color=labels,
        opacity=0.5,
        width=1200,
        height=800,
    )

    if cluster_centers is not None:
        fig.add_trace(
            go.Scatter3d(
                x=cluster_centers[:, 0],
                y=cluster_centers[:, 1],
                z=cluster_centers[:, 2],
                mode="markers",
                marker_symbol="x",
                hovertemplate="recency: %{x}, frequency: %{y}, monetary: %{z}",
                text="Cluster Center",
                name="Cluster Center",
            )
        )

    fig.show()


def plot_boxes(model_name, pred_df, labels, cluster_centers=None) -> None:
    pred_df["labels"] = labels

    fig = px.box(
        rfm_df,
        title=f"{ model_name } - Recency : days since last purchase",
        x="recency",
        color="labels",
        width=800,
    )
    fig.update_traces(boxmean="sd")
    fig.update_traces(notched=True)
    fig.show()

    fig = px.box(
        rfm_df,
        title=f"{ model_name } - Frequency : total number of purchases",
        x="frequency",
        color="labels",
        width=800,
    )
    fig.update_traces(boxmean="sd")
    fig.update_traces(notched=True)
    fig.show()

    fig = px.box(
        rfm_df,
        title=f"{ model_name } - Monetary : average purchase value",
        x="monetary",
        color="labels",
        width=800,
    )
    fig.update_traces(boxmean="sd")
    fig.update_traces(notched=True)
    fig.show()


### KMeans


In [ ]:
from sklearn.cluster import KMeans


results = process_model(
    model_class=KMeans,
    model_args={"random_state": 42},
    param_name="n_clusters",
    param_range=list(range(2, 11)),
    fit_df=scaled_rfm_df,
    pred_df=scaled_rfm_df,
)
plot_scores(results)


In [ ]:
best_result = results.sort_values(by="silhouette_score", ascending=False).iloc[
    0
]
models_results = models_results.append(best_result, ignore_index=True)

plot_clusters(
    model_name=best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(best_result["cluster_centers"])
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=best_result["model"],
    pred_df=rfm_df,
    labels=best_result["labels"],
)


In [ ]:
second_best_result = results.sort_values(
    by="silhouette_score", ascending=False
).iloc[1]
plot_clusters(
    model_name=second_best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=second_best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(
        second_best_result["cluster_centers"]
    )
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=second_best_result["model"],
    pred_df=rfm_df,
    labels=second_best_result["labels"],
)


### MiniBatchKMeans


In [ ]:
from sklearn.cluster import MiniBatchKMeans


results = process_model(
    model_class=MiniBatchKMeans,
    model_args={"random_state": 42},
    param_name="n_clusters",
    param_range=list(range(2, 11)),
    fit_df=scaled_rfm_df,
    pred_df=scaled_rfm_df,
)
plot_scores(results)


In [ ]:
best_result = results.sort_values(by="silhouette_score", ascending=False).iloc[
    0
]
models_results = models_results.append(best_result, ignore_index=True)

plot_clusters(
    model_name=best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(best_result["cluster_centers"])
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=best_result["model"],
    pred_df=rfm_df,
    labels=best_result["labels"],
)


In [ ]:
second_best_result = results.sort_values(
    by="silhouette_score", ascending=False
).iloc[1]
plot_clusters(
    model_name=second_best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=second_best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(
        second_best_result["cluster_centers"]
    )
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=second_best_result["model"],
    pred_df=rfm_df,
    labels=second_best_result["labels"],
)


### AffinityPropagation


In [ ]:
from sklearn.cluster import AffinityPropagation


results = process_model(
    model_class=AffinityPropagation,
    model_args={"random_state": 42},
    param_name="damping",
    param_range=[round(e, 3) for e in np.linspace(0.85, 0.99, 10)],
    fit_df=scaled_rfm_df.sample(frac=0.1, random_state=42),
    pred_df=scaled_rfm_df,
)
plot_scores(results)


In [ ]:
best_result = results.sort_values(by="silhouette_score", ascending=False).iloc[
    0
]
models_results = models_results.append(best_result, ignore_index=True)

plot_clusters(
    model_name=best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(best_result["cluster_centers"])
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=best_result["model"],
    pred_df=rfm_df,
    labels=best_result["labels"],
)


In [ ]:
second_best_result = results.sort_values(
    by="silhouette_score", ascending=False
).iloc[1]
plot_clusters(
    model_name=second_best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=second_best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(
        second_best_result["cluster_centers"]
    )
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=second_best_result["model"],
    pred_df=rfm_df,
    labels=second_best_result["labels"],
)


### AgglomerativeClustering


In [ ]:
from sklearn.cluster import AgglomerativeClustering

fit_pred_df = scaled_rfm_df.sample(frac=0.5, random_state=42)
results = process_model(
    model_class=AgglomerativeClustering,
    model_args={},
    param_name="n_clusters",
    param_range=list(range(2, 11)),
    fit_df=fit_pred_df,
    pred_df=fit_pred_df,
)
plot_scores(results)


In [ ]:
best_result = results.sort_values(by="silhouette_score", ascending=False).iloc[
    0
]
models_results = models_results.append(best_result, ignore_index=True)

plot_clusters(
    model_name=best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df.head(1000)),
        columns=fit_pred_df.columns,
    ),
    labels=best_result["labels"][:1000],
)
plot_boxes(
    model_name=best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df), columns=fit_pred_df.columns
    ),
    labels=best_result["labels"],
)


In [ ]:
second_best_result = results.sort_values(
    by="silhouette_score", ascending=False
).iloc[1]
plot_clusters(
    model_name=second_best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df.head(1000)),
        columns=fit_pred_df.columns,
    ),
    labels=second_best_result["labels"][:1000],
)
plot_boxes(
    model_name=second_best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df), columns=fit_pred_df.columns
    ),
    labels=second_best_result["labels"],
)


### MeanShift


In [ ]:
from sklearn.cluster import MeanShift


results = process_model(
    model_class=MeanShift,
    model_args={"n_jobs": -1},
    param_name="bandwidth",
    param_range=[round(e, 1) for e in np.linspace(0.5, 5, 10)],
    fit_df=scaled_rfm_df.sample(frac=0.1, random_state=42),
    pred_df=scaled_rfm_df,
)
plot_scores(results)


In [ ]:
best_result = results.sort_values(by="silhouette_score", ascending=False).iloc[
    0
]
models_results = models_results.append(best_result, ignore_index=True)

plot_clusters(
    model_name=best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(best_result["cluster_centers"])
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=best_result["model"],
    pred_df=rfm_df,
    labels=best_result["labels"],
)


In [ ]:
second_best_result = results.sort_values(
    by="silhouette_score", ascending=False
).iloc[1]
plot_clusters(
    model_name=second_best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=second_best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(
        second_best_result["cluster_centers"]
    )
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=second_best_result["model"],
    pred_df=rfm_df,
    labels=second_best_result["labels"],
)


### SpectralClustering


In [ ]:
from sklearn.cluster import SpectralClustering


fit_pred_df = scaled_rfm_df.sample(frac=0.2, random_state=42)
results = process_model(
    model_class=SpectralClustering,
    model_args={
        "random_state": 42,
        "n_jobs": -1,
    },
    param_name="n_clusters",
    param_range=list(range(2, 11)),
    fit_df=fit_pred_df,
    pred_df=fit_pred_df,
)
plot_scores(results)


In [ ]:
best_result = results.sort_values(by="silhouette_score", ascending=False).iloc[
    0
]
models_results = models_results.append(best_result, ignore_index=True)

plot_clusters(
    model_name=best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df.head(1000)),
        columns=fit_pred_df.columns,
    ),
    labels=best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(best_result["cluster_centers"])
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df), columns=fit_pred_df.columns
    ),
    labels=best_result["labels"],
)


In [ ]:
second_best_result = results.sort_values(
    by="silhouette_score", ascending=False
).iloc[1]

plot_clusters(
    model_name=second_best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df.head(1000)),
        columns=fit_pred_df.columns,
    ),
    labels=second_best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(
        second_best_result["cluster_centers"]
    )
    if second_best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=second_best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df), columns=fit_pred_df.columns
    ),
    labels=second_best_result["labels"],
)


### DBSCAN


In [ ]:
from sklearn.cluster import DBSCAN


fit_pred_df = scaled_rfm_df.sample(frac=0.5, random_state=42)
results = process_model(
    model_class=DBSCAN,
    model_args={
        "n_jobs": -1,
    },
    param_name="eps",
    param_range=[round(e, 2) for e in np.linspace(0.1, 1.5, 10)],
    fit_df=fit_pred_df,
    pred_df=fit_pred_df,
)
plot_scores(results)


In [ ]:
best_result = results.sort_values(by="silhouette_score", ascending=False).iloc[
    0
]
models_results = models_results.append(best_result, ignore_index=True)

plot_clusters(
    model_name=best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df.head(1000)),
        columns=fit_pred_df.columns,
    ),
    labels=best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(best_result["cluster_centers"])
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df), columns=fit_pred_df.columns
    ),
    labels=best_result["labels"],
)


In [ ]:
second_best_result = results.sort_values(
    by="silhouette_score", ascending=False
).iloc[1]

plot_clusters(
    model_name=second_best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df.head(1000)),
        columns=fit_pred_df.columns,
    ),
    labels=second_best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(
        second_best_result["cluster_centers"]
    )
    if second_best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=second_best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df), columns=fit_pred_df.columns
    ),
    labels=second_best_result["labels"],
)


### OPTICS


In [ ]:
from sklearn.cluster import OPTICS


fit_pred_df = scaled_rfm_df.sample(frac=0.5, random_state=42)
results = process_model(
    model_class=OPTICS,
    model_args={
        "n_jobs": -1,
    },
    param_name="min_samples",
    param_range=[round(e, 4) for e in np.linspace(0.001, 0.004, 10)],
    fit_df=fit_pred_df,
    pred_df=fit_pred_df,
)
plot_scores(results)


In [ ]:
best_result = results.sort_values(by="silhouette_score", ascending=False).iloc[
    0
]
models_results = models_results.append(best_result, ignore_index=True)

plot_clusters(
    model_name=best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df.head(1000)),
        columns=fit_pred_df.columns,
    ),
    labels=best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(best_result["cluster_centers"])
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df), columns=fit_pred_df.columns
    ),
    labels=best_result["labels"],
)


In [ ]:
second_best_result = results.sort_values(
    by="silhouette_score", ascending=False
).iloc[1]

plot_clusters(
    model_name=second_best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df.head(1000)),
        columns=fit_pred_df.columns,
    ),
    labels=second_best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(
        second_best_result["cluster_centers"]
    )
    if second_best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=second_best_result["model"],
    pred_df=pd.DataFrame(
        scaler.inverse_transform(fit_pred_df), columns=fit_pred_df.columns
    ),
    labels=second_best_result["labels"],
)


### Birch


In [ ]:
from sklearn.cluster import Birch


results = process_model(
    model_class=Birch,
    model_args={},
    param_name="n_clusters",
    param_range=list(range(2, 11)),
    fit_df=scaled_rfm_df.sample(frac=0.2, random_state=42),
    pred_df=scaled_rfm_df,
)
plot_scores(results)


In [ ]:
best_result = results.sort_values(by="silhouette_score", ascending=False).iloc[
    0
]
models_results = models_results.append(best_result, ignore_index=True)

plot_clusters(
    model_name=best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(best_result["cluster_centers"])
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=best_result["model"],
    pred_df=rfm_df,
    labels=best_result["labels"],
)


In [ ]:
second_best_result = results.sort_values(
    by="silhouette_score", ascending=False
).iloc[1]
plot_clusters(
    model_name=second_best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=second_best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(
        second_best_result["cluster_centers"]
    )
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=second_best_result["model"],
    pred_df=rfm_df,
    labels=second_best_result["labels"],
)


### GMM (Gaussian Mixture)


In [ ]:
from sklearn.mixture import GaussianMixture


results = process_model(
    model_class=GaussianMixture,
    model_args={
        "random_state": 42,
    },
    param_name="n_components",
    param_range=list(range(2, 11)),
    fit_df=scaled_rfm_df,
    pred_df=scaled_rfm_df,
)
plot_scores(results)


In [ ]:
best_result = results.sort_values(by="silhouette_score", ascending=False).iloc[
    0
]
models_results = models_results.append(best_result, ignore_index=True)

plot_clusters(
    model_name=best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(best_result["cluster_centers"])
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=best_result["model"],
    pred_df=rfm_df,
    labels=best_result["labels"],
)


In [ ]:
second_best_result = results.sort_values(
    by="silhouette_score", ascending=False
).iloc[1]
plot_clusters(
    model_name=second_best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=second_best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(
        second_best_result["cluster_centers"]
    )
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=second_best_result["model"],
    pred_df=rfm_df,
    labels=second_best_result["labels"],
)


### Bayesian Gaussian Mixture


In [ ]:
from sklearn.mixture import BayesianGaussianMixture


results = process_model(
    model_class=BayesianGaussianMixture,
    model_args={
        "random_state": 42,
    },
    param_name="n_components",
    param_range=list(range(2, 11)),
    fit_df=scaled_rfm_df,
    pred_df=scaled_rfm_df,
)
plot_scores(results)


In [ ]:
best_result = results.sort_values(by="silhouette_score", ascending=False).iloc[
    0
]
models_results = models_results.append(best_result, ignore_index=True)

plot_clusters(
    model_name=best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(best_result["cluster_centers"])
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=best_result["model"],
    pred_df=rfm_df,
    labels=best_result["labels"],
)


In [ ]:
second_best_result = results.sort_values(
    by="silhouette_score", ascending=False
).iloc[1]
plot_clusters(
    model_name=second_best_result["model"],
    pred_df=rfm_df.head(1000),
    labels=second_best_result["labels"][:1000],
    cluster_centers=scaler.inverse_transform(
        second_best_result["cluster_centers"]
    )
    if best_result["cluster_centers"] is not None
    else None,
)
plot_boxes(
    model_name=second_best_result["model"],
    pred_df=rfm_df,
    labels=second_best_result["labels"],
)


### Comparison


In [ ]:
models_results.sort_values(by="silhouette_score", ascending=False)
